In [65]:
# house_prices
# instanciar o Kfold

# instanciar modelo 1 -> Elasticnet alpha=1.3

#rodam cross_validate

# instanciar modelo 2 -> Elasticnet alpha=1.5

#rodam cross_validate

# instanciar modelo 3 -> Elasticnet alpha=0.9

#rodam cross_validate



#comparar metrica de erro entre modelo 1, 2 e 3 e escolher o melhor

In [66]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
from ml_utils import *

In [68]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [69]:
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold

In [70]:
from sklearn.model_selection import cross_validate

In [71]:
df = pd.read_csv('house_prices.csv')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [72]:
X = df.drop(columns=["Id", "SalePrice"])
y = df["SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [73]:
X_train_model = X_train.select_dtypes(include=np.number).dropna(axis="columns")
X_test_model = X_test.loc[:, X_train_model.columns]

## Overfitting

In [74]:
pf = PolynomialFeatures(degree=2, include_bias=False).fit(X_train_model)
        
X_train = pf.transform(X_train_model)
X_test = pf.transform(X_test_model)

In [75]:
reg_lin_pt1_pt2(X_train, y_train, X_test, y_test, 
                    plot=False, scale_mms=False, 
                    train_metrics=True, dist_resids=True)


Métricas de avaliação (dados de treino):

R^2: 0.96
MAE: 10528.16
RMSE: 14982.18
MAPE: 6.41%

Distribuição dos resíduos de treino:

count    1.168000e+03
mean    -7.466911e-07
std      1.498859e+04
min     -1.085426e+05
25%     -8.016138e+03
50%      4.929192e-01
75%      8.172938e+03
max      9.457485e+04
Name: SalePrice, dtype: float64

##################################################

Métricas de avaliação (dados de teste):

R^2: 0.65
MAE: 27307.25
RMSE: 51512.62
MAPE: 17.82%

Distribuição dos resíduos de teste:

count       292.000000
mean       1248.446923
std       51585.897163
min     -353345.126807
25%      -13377.276551
50%        1166.358874
75%       15438.545932
max      318725.834764
Name: SalePrice, dtype: float64


LinearRegression()

## Linear Regression Regularization

In [76]:
reg_lin_poly_features_regularized(X_train_model, y_train, X_test_model, y_test, 
                    
                                      deg=3,
                                      type_regularization="l1", alpha=50,
                                      iter_max=2000,
                                      plot=False, scale_mms=False, 
                                      train_metrics=True, 
                                      dist_resids=False,
                                      plot_model=False)

Modelo com espaço de features transformado!

Número de features original: 33
Número de features após o transformer: 7139


Métricas de avaliação (dados de treino):

R^2: 0.93
MAE: 14550.68
RMSE: 20633.57
MAPE: 8.53%

##################################################

Métricas de avaliação (dados de teste):

R^2: 0.89
MAE: 18155.80
RMSE: 28770.64
MAPE: 10.80%


(Lasso(alpha=50, max_iter=2000),
 {'r2': 0.9286209087963528,
  'mae': 14550.675382250174,
  'rmse': 20633.56938453419,
  'mape': 0.0853363302936373},
 {'r2': 0.8920841444764087,
  'mae': 18155.79649067913,
  'rmse': 28770.641542962192,
  'mape': 0.10798075175270948})

## CV

In [82]:
# Create a KFold object
kf = KFold(n_splits=6, shuffle=True, random_state=42)



# Compute 6-fold cross-validation scores
cv_scores = cross_val_score(LinearRegression(), X_train_model, y_train, cv=kf, scoring='r2')

# Print scores
print(cv_scores)
print(np.mean(cv_scores))
print(np.std(cv_scores))

[0.83998994 0.80074845 0.75857116 0.54436684 0.8197479  0.85153284]
0.7691595216372491
0.10488344439945024


In [84]:
kfS = StratifiedKFold(n_splits=6, shuffle=True, random_state=42)



# Compute 6-fold cross-validation scores
cv_scores = cross_val_score(LinearRegression(), X_train_model, y_train, cv=kfS, scoring='r2')

# Print scores
print(cv_scores)
print(np.mean(cv_scores))
print(np.std(cv_scores))

[0.84047638 0.8196953  0.79673866 0.46485044 0.82073895 0.70433942]
0.7411398582512079
0.13112084751796865


C:\Users\Usuario\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=6.
  warnings.warn(


In [85]:
# Compute 6-fold cross-validation scores
cv_scores = cross_val_score(Lasso(alpha=1.3), X_train_model, y_train, cv=kf, scoring='r2')

# Print scores
print(cv_scores)
print(np.mean(cv_scores))
print(np.std(cv_scores))

[0.84000095 0.80076615 0.75857942 0.54432932 0.8197902  0.85154347]
0.7691682523044889
0.1049036258473521


In [80]:
cv_scores = cross_val_score(Lasso(alpha=1.5), X_train_model, y_train, cv=kf, scoring='r2')

# Print scores
print(cv_scores)
print(np.mean(cv_scores))
print(np.std(cv_scores))

[0.83430496 0.73615063 0.61880309 0.80546939 0.84405008]
0.7677556309274033
0.08349577169211327


In [81]:
cv_scores = cross_val_score(Lasso(alpha=.9), X_train_model, y_train, cv=kf, scoring='r2')

# Print scores
print(cv_scores)
print(np.mean(cv_scores))
print(np.std(cv_scores))

[0.83430254 0.73614259 0.61881535 0.80545882 0.84403548]
0.7677509575393584
0.08348799927971876


In [86]:
kf = KFold(n_splits=6, shuffle=True, random_state=42)



# Compute 6-fold cross-validation scores
cv_scores = cross_val_score(ElasticNet(alpha=1.3), X_train_model, y_train, cv=kf, scoring='r2')

# Print scores
print(cv_scores)
print(np.mean(cv_scores))
print(np.std(cv_scores))

C:\Users\Usuario\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.850e+11, tolerance: 5.852e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\Usuario\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.383e+11, tolerance: 5.617e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\Usuario\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.463e+11, to

[0.83431837 0.79445308 0.75008546 0.49625996 0.81193814 0.8521986 ]
0.7565422676688812
0.12075716613785131


In [87]:
kf = KFold(n_splits=6, shuffle=True, random_state=42)



# Compute 6-fold cross-validation scores
cv_scores = cross_val_score(ElasticNet(alpha=1.5), X_train_model, y_train, cv=kf, scoring='r2')

# Print scores
print(cv_scores)
print(np.mean(cv_scores))
print(np.std(cv_scores))

C:\Users\Usuario\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.922e+11, tolerance: 5.852e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\Usuario\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.453e+11, tolerance: 5.617e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\Usuario\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.530e+11, to

[0.83263808 0.79285448 0.74776959 0.49165029 0.80931303 0.85083475]
0.7541767042451766
0.12178951691582524


In [88]:
kf = KFold(n_splits=6, shuffle=True, random_state=42)



# Compute 6-fold cross-validation scores
cv_scores = cross_val_score(ElasticNet(alpha=.9), X_train_model, y_train, cv=kf, scoring='r2')

# Print scores
print(cv_scores)
print(np.mean(cv_scores))
print(np.std(cv_scores))

C:\Users\Usuario\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.670e+11, tolerance: 5.852e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\Usuario\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.205e+11, tolerance: 5.617e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\Usuario\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.292e+11, to

[0.83788533 0.7978817  0.75503977 0.50721193 0.81763265 0.85493491]
0.761764382838618
0.11814048375859391
